In [1]:
import pandas as pd
import numpy as np

In [2]:
######## PART 1 ########

In [3]:
neg_class_mean_vector = np.array(pd.read_csv('./hwk2_datasets/DS1_m_0.txt', header=None)).T
pos_class_mean_vector = np.array(pd.read_csv('./hwk2_datasets/DS1_m_1.txt', header=None)).T
cov_matrix = np.array(pd.read_csv('./hwk2_datasets/DS1_Cov.txt', header=None))

In [4]:
neg_class_mean_vector.shape

(20, 1)

In [5]:
pos_class_mean_vector.shape

(20, 1)

In [6]:
cov_matrix.shape

(20, 20)

In [7]:
neg_class_mean_vector

array([[1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3],
       [1.3]])

In [8]:
pos_class_mean_vector

array([[2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.]])

In [9]:
cov_matrix

array([[7.92747613, 5.48849335, 6.18812365, 5.17766549, 5.84803095,
        6.15325514, 4.6623167 , 5.45500286, 4.98574312, 5.18758586,
        3.9436397 , 5.26132514, 7.09637106, 5.99810181, 6.0932167 ,
        5.99518462, 5.87317499, 5.70538764, 5.59907089, 5.91786273],
       [5.48849335, 6.81598145, 5.34490996, 4.31020106, 5.41645238,
        5.53995027, 4.30811201, 3.93658438, 4.11429237, 4.98428213,
        3.34393835, 4.59050685, 5.85714703, 5.07764986, 5.41356597,
        5.19974264, 5.60959615, 5.09170926, 5.31814733, 5.27567458],
       [6.18812365, 5.34490996, 7.322937  , 4.87361613, 5.78994181,
        6.60647915, 4.61175555, 4.83989875, 4.86905732, 5.05339559,
        3.26344459, 4.68117364, 6.36913218, 5.17240371, 6.107158  ,
        5.99640412, 6.24914837, 4.99487173, 4.72323191, 5.0810024 ],
       [5.17766549, 4.31020106, 4.87361613, 5.77060116, 5.26312518,
        4.50414113, 3.70666948, 4.39831182, 3.37569889, 4.20589158,
        2.74695007, 4.1714241 , 5.86605885, 4

In [10]:
pos_class_examples = np.random.multivariate_normal(pos_class_mean_vector, cov_matrix, size=2000)

ValueError: mean must be 1 dimensional